In [1]:
library("survival")
library("survminer")
library("ranger")
library("ggplot2")
library("dplyr")
library("ggfortify")

Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Loading required package: ggpubr
Loading required package: magrittr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
df <- read.csv("Final_NHANES.csv")

In [3]:
for (i in 1:nrow(df)){
    df$SEQN[i] = i
}

In [4]:
levels(df$mortstat) <- c(0,1)

In [5]:
head(df)

SEQN,Gender,Age,Race1,Education,MaritalStatus,HHIncomeMid,Poverty,HomeRooms,HomeOwn,...,UrineFlow1,Diabetes,HealthGen,DaysPhysHlthBad,DaysMentHlthBad,SleepHrsNight,SleepTrouble,PhysActive,Smoke100,mortstat
1,male,34.00000,Race1_White,High School,Married,30000.00,1.360000,6.000000,HomeOwn_Own,...,1.7017764,Diabetes_No,Good,0.000000,15.0000000,4.000000,SleepTrouble_Yes,PhysActive_No,Smoke100_Yes,0
2,male,34.00000,Race1_White,High School,Married,30000.00,1.360000,6.000000,HomeOwn_Own,...,1.7017764,Diabetes_No,Good,0.000000,15.0000000,4.000000,SleepTrouble_Yes,PhysActive_No,Smoke100_Yes,0
3,male,34.00000,Race1_White,High School,Married,30000.00,1.360000,6.000000,HomeOwn_Own,...,1.7017764,Diabetes_No,Good,0.000000,15.0000000,4.000000,SleepTrouble_Yes,PhysActive_No,Smoke100_Yes,0
4,male,4.00000,Race1_Other,College Grad,NeverMarried,22500.00,1.070000,9.000000,HomeOwn_Own,...,0.3680715,Diabetes_No,Vgood,-4.463701,0.2140656,7.926828,SleepTrouble_No,PhysActive_Yes,Smoke100_No,0
5,female,38.82787,Race1_White,Some College,Married,55898.88,2.769805,6.220442,HomeOwn_Own,...,0.9426834,Diabetes_No,Good,2.817233,4.0134528,7.003282,SleepTrouble_No,PhysActive_Yes,Smoke100_No,0
6,female,38.82787,Race1_White,Some College,Married,55898.88,2.769805,6.220442,HomeOwn_Own,...,0.9426834,Diabetes_No,Good,2.817233,4.0134528,7.003282,SleepTrouble_No,PhysActive_Yes,Smoke100_No,0


In [6]:
covariates <- c("BPSysAve", "DirectChol", "BPDiaAve", "Weight", "PhysActive", "Poverty", "TotChol", "UrineFlow1", "DaysMentHlthBad", "UrineVol1", "DaysPhysHlthBad", "BMI", "HHIncomeMid", "SleepHrsNight", "Pulse", "Gender", "Diabetes", "Smoke100", "SleepTrouble")
univ_formulas <- sapply(covariates,
                        function(x) as.formula(paste('Surv(Age, mortstat)~', x)))
                        
univ_models <- lapply( univ_formulas, function(x){coxph(x, data = df, id = SEQN)})
# Extract data 
univ_results <- lapply(univ_models,
                       function(x){ 
                          x <- summary(x)
                          p.value<-signif(x$wald["pvalue"], digits=2)
                          wald.test<-signif(x$wald["test"], digits=2)
                          beta<-signif(x$coef[1], digits=2);#coeficient beta
                          HR <-signif(x$coef[2], digits=2);#exp(beta)
                          HR.confint.lower <- signif(x$conf.int[,"lower .95"], 2)
                          HR.confint.upper <- signif(x$conf.int[,"upper .95"],2)
                          HR <- paste0(HR, " (", 
                                       HR.confint.lower, "-", HR.confint.upper, ")")
                          res<-c(beta, HR, wald.test, p.value)
                          names(res)<-c("beta", "HR (95% CI for HR)", "wald.test", 
                                        "p.value")
                          return(res)
                          #return(exp(cbind(coef(x),confint(x))))
                         })
res <- t(as.data.frame(univ_results, check.names = FALSE))
as.data.frame(res)

,beta,HR (95% CI for HR),wald.test,p.value
BPSysAve,-0.027,0.97 (0.97-0.98),48,4.5e-12
DirectChol,-0.36,0.7 (0.41-1.2),1.8,0.18
BPDiaAve,-0.03,0.97 (0.97-0.98),150,1.9e-33
Weight,-0.022,0.98 (0.97-0.98),98,3.9e-23
PhysActive,-0.25,0.78 (0.57-1.1),2.5,0.11
Poverty,-0.45,0.63 (0.56-0.72),50,1.5e-12
TotChol,-0.54,0.58 (0.51-0.66),73,1.5e-17
UrineFlow1,-0.7,0.5 (0.34-0.74),12,0.00049
DaysMentHlthBad,-0.026,0.97 (0.94-1),1.8,0.18
UrineVol1,-0.0046,1 (0.99-1),11,0.00077


In [7]:
new_df <- df[,c("SEQN", "Age","BPSysAve", "DirectChol", "BPDiaAve", "Weight", "PhysActive" , "Poverty", "TotChol", "UrineFlow1", "DaysMentHlthBad", "UrineVol1", "DaysPhysHlthBad", "BMI", "HHIncomeMid", "SleepHrsNight", "Pulse", "Gender", "Diabetes", "Smoke100", "SleepTrouble", "mortstat")]

In [8]:
head(new_df)

SEQN,Age,BPSysAve,DirectChol,BPDiaAve,Weight,PhysActive,Poverty,TotChol,UrineFlow1,...,DaysPhysHlthBad,BMI,HHIncomeMid,SleepHrsNight,Pulse,Gender,Diabetes,Smoke100,SleepTrouble,mortstat
1,34.00000,113.00000,1.290000,85.00000,87.40000,PhysActive_No,1.360000,3.490000,1.7017764,...,0.000000,32.2200,30000.00,4.000000,70.00000,male,Diabetes_No,Smoke100_Yes,SleepTrouble_Yes,0
2,34.00000,113.00000,1.290000,85.00000,87.40000,PhysActive_No,1.360000,3.490000,1.7017764,...,0.000000,32.2200,30000.00,4.000000,70.00000,male,Diabetes_No,Smoke100_Yes,SleepTrouble_Yes,0
3,34.00000,113.00000,1.290000,85.00000,87.40000,PhysActive_No,1.360000,3.490000,1.7017764,...,0.000000,32.2200,30000.00,4.000000,70.00000,male,Diabetes_No,Smoke100_Yes,SleepTrouble_Yes,0
4,4.00000,91.25459,1.532677,46.95456,17.00000,PhysActive_Yes,1.070000,3.882090,0.3680715,...,-4.463701,15.3000,22500.00,7.926828,80.02207,male,Diabetes_No,Smoke100_No,SleepTrouble_No,0
5,38.82787,116.79930,1.374860,66.33432,73.51088,PhysActive_Yes,2.769805,4.842883,0.9426834,...,2.817233,26.9935,55898.88,7.003282,74.00662,female,Diabetes_No,Smoke100_No,SleepTrouble_No,0
6,38.82787,116.79930,1.374860,66.33432,73.51088,PhysActive_Yes,2.769805,4.842883,0.9426834,...,2.817233,26.9935,55898.88,7.003282,74.00662,female,Diabetes_No,Smoke100_No,SleepTrouble_No,0


In [9]:
res.cox <- coxph(Surv(Age, mortstat) ~ BPSysAve + DirectChol + BPDiaAve + Weight + PhysActive + Poverty + TotChol + UrineFlow1 + DaysMentHlthBad + UrineVol1 + DaysPhysHlthBad + BMI + HHIncomeMid + SleepHrsNight + Pulse + Gender + Diabetes + Smoke100 + SleepTrouble, new_df, id=SEQN)
summary(res.cox)

Call:
coxph(formula = Surv(Age, mortstat) ~ BPSysAve + DirectChol + 
    BPDiaAve + Weight + PhysActive + Poverty + TotChol + UrineFlow1 + 
    DaysMentHlthBad + UrineVol1 + DaysPhysHlthBad + BMI + HHIncomeMid + 
    SleepHrsNight + Pulse + Gender + Diabetes + Smoke100 + SleepTrouble, 
    data = new_df, id = SEQN)

  n= 23514, number of events= 880 

         coef  exp(coef)   se(coef)  robust se      z Pr(>|z|)    
1  -1.524e-02  9.849e-01  3.198e-03  3.626e-03 -4.203 2.63e-05 ***
2  -3.489e-01  7.054e-01  1.958e-01  2.527e-01 -1.381 0.167386    
3  -8.361e-03  9.917e-01  4.162e-03  4.481e-03 -1.866 0.062041 .  
4  -3.287e-02  9.677e-01  8.608e-03  1.066e-02 -3.084 0.002044 ** 
5  -5.453e-01  5.797e-01  1.536e-01  1.777e-01 -3.068 0.002153 ** 
6  -3.770e-01  6.859e-01  1.039e-01  1.276e-01 -2.954 0.003135 ** 
7  -9.512e-02  9.093e-01  6.454e-02  8.263e-02 -1.151 0.249689    
8  -5.372e-02  9.477e-01  1.191e-01  1.919e-01 -0.280 0.779535    
9  -2.188e-02  9.784e-01  8.934e-03  1.044e

In [10]:
#Error here!
cox_fit <- survfit(res.cox)

ERROR: Error in hazard %*% diag(risk2[i, ]): argumentos não compatíveis
